In [26]:
import torch
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, CODECONTESTS_COT_CONFIG, SMOL_LM_135M

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
cfg = CODECONTESTS_COT_CONFIG
cfg.notebook_mode = True
cfg.model_id_or_path = SMOL_LM_135M
wrapper = TrainerWrapper(cfg)

2025-01-09 21:33:44.316 | INFO     | trl_wrapper.trainer_wrapper:__init__:183 - Adding special tokens: ['<thought>', '</thought>', '<solution>', '</solution>']


In [28]:
wrapper.init_model()

2025-01-09 21:33:47.077 | INFO     | trl_wrapper.trainer_wrapper:init_model:199 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x146e0dfb9a10>>
Traceback (most recent call last):
  File "/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
2025-01-09 21:33:50.731 | INFO     | trl_wrapper.trainer_wrapper:init_model:210 - Resizing token embeddings for model to 49156
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [40]:
wrapper.init_data_module()

2025-01-09 21:40:36.027 | INFO     | model.utils:__init__:110 - Cache dir: ../dataset_caches/conversation_raw_data_module
2025-01-09 21:40:36.029 | INFO     | dataset.conversation:__init__:40 - Cache dir: ../dataset_caches/conversation_raw_data_module/HuggingFaceTB/SmolLM2-135M-Instruct/openo1_sft_formatted_thoughts_conversations.parquet
2025-01-09 21:40:36.029 | INFO     | model.utils:setup:128 - Loading dataset for stage fit
2025-01-09 21:40:36.030 | INFO     | model.utils:setup:131 - Processing dataset for stage fit, workers: 1, cache dir ../dataset_caches/conversation_raw_data_module/HuggingFaceTB/SmolLM2-135M-Instruct/openo1_sft_formatted_thoughts_conversations.parquet, using cache: False
2025-01-09 21:40:36.031 | INFO     | dataset.conversation:load_dataset:45 - Loading dataset
2025-01-09 21:40:36.092 | INFO     | model.utils:setup:139 - Train dataset size: 14445 Val dataset size: 1605


True


Map: 100%|██████████| 1605/1605 [00:02<00:00, 647.54 examples/s]


In [38]:
# Tokenizer dev
first_batch = wrapper.data_module.train_dataset[0]
tokenized = wrapper.data_module._tokenize_conversation(first_batch['conversation'])
for k in tokenized.keys():
    tokenized[k] = torch.as_tensor(tokenized[k], dtype=torch.long)
print(tokenized['labels'].tolist())
wrapper.data_module.visualize_sample(tokenized)

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

<|im_start|>\n<|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 09 Jan 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write an IF ELSE statement in SQL to check if a value is greater than 10. If it is, set the value to 10. If it is 
not, leave it as is. Additionally, handle the scenario when the value is exactly 10 by setting it to 11.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

<thought>Alright, I need to write an IF ELSE statement in SQL to handle some specific conditions for a value. The 
conditions are: if the value is greater than 10, set it to 10; if it's not greater than 10, leave it as is; and if 
the value is exactly 10, set it to 11. Hmm, that's interesting because the third condition overlaps with the first 
two. I need to ensure that when the value is exactly 10, it doesn't conflict with the other conditions.</thought>
<thought>First, let's think about how SQL handles IF ELSE statements. Depending on the SQL dialect, the syntax 
might vary. For example, in T-SQL (used by SQL Server), you can use IF ELSE within a stored procedure or a batch of
SQL statements. However, if I'm writing this within a SELECT statement, I might need to use the CASE expression 
instead.</thought>
<thought>Wait, does the user want an IF ELSE block within a procedural context, or are they looking for something 
that can be used directly in a query? The question doesn't specify, so I should probably consider both 
scenarios.</thought>
<thought>Starting with the procedural approach using IF ELSE in T-SQL:</thought>
<thought>```
IF (@value > 10)
BEGIN
    SET @value = 10
END
ELSE IF (@value = 10)
BEGIN
    SET @value = 11
END
ELSE
BEGIN
    -- Leave it as is
END
```</thought>
<thought>But, I notice a logical issue here. If the value is exactly 10, it doesn't satisfy `@value > 10`, so it 
would go to the ELSE IF condition. That seems fine. However, the user wants to handle the exactly 10 case 
separately by setting it to 11. So, the order of these conditions matters.</thought>
<thought>Alternatively, I could reorder the conditions to check if the value is exactly 10 first:</thought>
<thought>```
IF (@value = 10)
BEGIN
    SET @value = 11
END
ELSE IF (@value > 10)
BEGIN
    SET @value = 10
END
ELSE
BEGIN
    -- Leave it as is
END
```</thought>
<thought>This way, if the value is exactly 10, it gets set to 11, and only values greater than 10 get set to 10. 
That seems more logical and prevents any overlap issues.</thought>
<thought>Now, if I consider writing this within a SELECT statement using CASE, which is more common for inline 
operations:</thought>
<thought>```
SELECT 
    CASE 
        WHEN value > 10 THEN 10
        WHEN value = 10 THEN 11
        ELSE value
    END as adjusted_value
FROM table_name;
```</thought>
<thought>This CASE statement checks the conditions in order. First, it checks if the value is greater than 10, and 
sets it to 10. Then, it checks if the value is exactly 10 and sets it to 11. If neither condition is met, it leaves
the value as is. This approach is clean and works well within a query.</thought>
<thought>But wait, is there a possibility of conflicts or unexpected behaviors? For instance, what if the value is 
NULL? Should I handle that as well? The question doesn't specify, so I'll assume that the value is always a valid 
number.</thought>
<thought>Also, depending on the SQL dialect, the syntax might need slight adjustments. For example, some databases 
might require different keywords or functions. However, the general logic remains the same.</thought>
<thought>Another thought: if this logic is to be applied to multiple rows, the CASE statement within the SELECT 
would be more efficient than using procedural IF ELSE statements, which are row-by-row operations.</thought>
<thought>Moreover, if I need to update existing records based on this logic, I could use the CASE statement within 
an UPDATE statement like this:</thought>
<thought>```
UPDATE tab

In [39]:
# mock_labels = first_batch["labels"].clone().detach()
# mock_labels = torch.where(mock_labels == -100, wrapper.data_module.tokenizer.pad_token_id, mock_labels)
# mock_ids = first_batch["input_ids"].clone().detach()
for batch in wrapper.data_module.val_dataloader():
    print(batch["input_ids"].shape)
    print(batch["labels"].shape)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


KeyError: 'input_ids'

In [ ]:
wrapper.init_trainer()

In [ ]:
wrapper.tokenizer

In [ ]:
wrapper.train()